In [36]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pandas as pd
df = pd.read_csv("/content/gdrive/MyDrive/PGSS2023/PGSS2023--ClassDrive/Week02/IMDB Dataset.csv")

In [4]:
# Distribution of each type of review
df["sentiment"].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Cleanse text data





In [6]:
# Convert all words to lowercase
df["review"] = df["review"].str.lower()

# Remove HTML tags using Beautiful soup
from bs4 import BeautifulSoup
df["review"] = df["review"].apply(lambda review: BeautifulSoup(review, "lxml").text)

# Remove urls
# From https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
import re
df["review"] = df["review"].apply(lambda review: re.sub(r'^https?:\/\/.*[\r\n]*', '', review, flags=re.MULTILINE))

df.head()

<ipython-input-6-6e751ea85957>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df["review"] = df["review"].apply(lambda review: BeautifulSoup(review, "lxml").text)


,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [7]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.0 MB/s eta 0:00:00


In [8]:
# Remove Contractions
import contractions
df["review"] = df["review"].apply(lambda review: contractions.fix(review))

Classify text after converting response variable

In [9]:
#Create Word Cloud
from wordcloud import WordCloud, STOPWORDS

all_reviews = " "
positive_reviews = " "
negative_reviews =  ""

stopwords = set(STOPWORDS)


positive_df = df[df["sentiment"]  == "positive"]
negative_df = df[df["sentiment"]  == "negative"]

for review in positive_df.review:
    # Tokenize the review
    tokens = review.split()

    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()

    all_reviews += " ".join(tokens)+" "
    positive_reviews  += " ".join(tokens)+" "

for review in negative_df.review:
    # Tokenize the review
    tokens = review.split()

    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()

    all_reviews += " ".join(tokens)+" "
    negative_reviews  += " ".join(tokens)+" "


KeyboardInterrupt: ignored

In [ ]:
type(all_reviews)

In [ ]:
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(all_reviews)


In [ ]:
# plot the WordCloud image

import matplotlib.pyplot as plt

plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.title("Word Cloud from All Reviews")
plt.show()


In [ ]:
import re
from collections import Counter
import matplotlib.pyplot as plt


def create_barplot(string, k):
    # Step 1: Convert string to list of words
    words = string.split()

    # Step 2: Remove punctuation and special characters
    words = [re.sub(r'[^\w\s]', '', word) for word in words]

    # Step 3: Convert all words to lowercase
    words = [word.lower() for word in words]

    # Step 4: Count frequency of each word using dictionary
    word_counts = Counter(words)

    # Step 5: Sort dictionary by value in descending order and limit to top k
    sorted_word_counts = dict(sorted(word_counts.items(), key=lambda item: item[1], reverse=True)[:k])

    # Step 6: Create barplot using matplotlib
    plt.bar(sorted_word_counts.keys(), sorted_word_counts.values())
    plt.xticks(rotation=90)
    plt.xlabel('Words')
    plt.ylabel('Frequency')
    plt.title(f'Top {k} Most Frequent Words')
    plt.show()

k = 20

create_barplot(all_reviews, k)

In [ ]:
k = 20

create_barplot(negative_reviews, k)

In [ ]:
k = 20

create_barplot(positive_reviews, k)

In [ ]:
# Define a function to count the number of words in a string
def count_words(text):
    return len(text.split())

# Apply the function to the reviews column using apply method
df['num_words'] = df['review'].apply(count_words)

# Create a new column in the dataframe with the number of words
df.head()

In [ ]:
import seaborn as sns

# Create a histogram using seaborn's histplot function
sns.histplot(data=df, x='num_words', bins=10)

# Display the plots
plt.show()

In [ ]:

# Create a KDE plot using seaborn's kdeplot function
sns.kdeplot(data=df, x='num_words')


In [ ]:
#  Create a grouped boxplot using seaborn's boxplot function
sns.boxplot(data=df, x='sentiment', y='num_words')

# Display the plot
plt.show()

## Create new numerical features in addition to the num_words, using sentence transformers to embed the text into fixed-length numerical representations

In [10]:
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=6e5d251127908e5cd9f74e29eb8626e53db3b8c0c1314d200fd9a3351f053d2c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [68]:
# Create a stratified random sample of df
strata = df['sentiment'].unique()

# Calculate the size of the sample for each stratum
sample_size = 1670
sample_sizes = {s: sample_size for s in strata}

# Use pandas' groupby method to group the dataframe by the sentiment column
groups = df.groupby('sentiment')

# Use pandas' apply method to sample each group based on the size of the sample for that stratum
samples = groups.apply(lambda x: x.sample(n=sample_sizes[x.name])).reset_index(drop=True)

# Concatenate the sampled groups into a single dataframe
sampled_df = pd.concat([samples])

sampled_df

,review,sentiment
0,there are so many incorrect statements in this...,negative
1,this is just about one of the dumbest things i...,negative
2,"the only reason ""the norliss tapes"" deserves a...",negative
3,this is one of them movies that has a awesome ...,negative
4,my grandad gave me this dvd. with friends like...,negative
...,...,...
3335,this film is like an allegory of the gospel. i...,positive
3336,i saw dark harbor at the '98 seattle film fest...,positive
3337,i caught this film at a test screening. was ve...,positive
3338,an absolute classic of 80's scare flix. this o...,positive


In [69]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence transformer model
# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('bert-base-nli-mean-tokens', device='cuda')

# Define a function to embed the text using the sentence transformer model
def embed_text(text):
    embeddings = model.encode(text)
    return embeddings.tolist()

# Apply the function to the reviews column using apply method
sampled_df['embeddings'] = sampled_df['review'].apply(embed_text)

sampled_df

,review,sentiment,embeddings
0,there are so many incorrect statements in this...,negative,"[0.27332693338394165, 0.7375638484954834, 0.74..."
1,this is just about one of the dumbest things i...,negative,"[-0.49211403727531433, 1.1527941226959229, -0...."
2,"the only reason ""the norliss tapes"" deserves a...",negative,"[-0.13038009405136108, 0.6600782871246338, 0.5..."
3,this is one of them movies that has a awesome ...,negative,"[-0.061912328004837036, 0.31824058294296265, 0..."
4,my grandad gave me this dvd. with friends like...,negative,"[-0.4051836133003235, 0.6496313810348511, 0.27..."
...,...,...,...
3335,this film is like an allegory of the gospel. i...,positive,"[-0.27085134387016296, 0.8734394311904907, 0.1..."
3336,i saw dark harbor at the '98 seattle film fest...,positive,"[-0.26888149976730347, 0.34472984075546265, 0...."
3337,i caught this film at a test screening. was ve...,positive,"[-0.6633926033973694, 0.3935912251472473, 0.23..."
3338,an absolute classic of 80's scare flix. this o...,positive,"[-0.13873495161533356, 0.5045498013496399, 0.2..."


In [70]:
df_embeddings = pd.DataFrame(list(sampled_df['embeddings']))

sampled_df = pd.concat([sampled_df, df_embeddings], axis=1)
sampled_df.drop(['embeddings'], axis=1, inplace=True)

# df_model = sampled_df[['num_words', 0, 1, ..., 767]]

sampled_df

,review,sentiment,0,1,2,3,4,5,6,7,...,758,759,760,761,762,763,764,765,766,767
0,there are so many incorrect statements in this...,negative,0.273327,0.737564,0.740481,0.235589,-0.549669,-1.168450,1.045666,0.050999,...,-0.504865,-0.335349,-1.372853,-2.010983,-0.313539,-0.939627,0.359824,0.658613,0.300944,0.361359
1,this is just about one of the dumbest things i...,negative,-0.492114,1.152794,-0.250474,0.139126,-0.032523,-0.892238,1.060019,0.233915,...,-0.207601,-0.501395,-1.164164,-1.465709,0.172248,-0.187147,0.014653,0.056565,0.893500,0.207951
2,"the only reason ""the norliss tapes"" deserves a...",negative,-0.130380,0.660078,0.567430,0.237133,-0.103683,-0.940961,1.543659,-0.179284,...,-0.647385,-0.845036,-1.175116,-1.531967,-0.392439,-0.758413,-0.250309,0.167722,-0.209265,-0.121306
3,this is one of them movies that has a awesome ...,negative,-0.061912,0.318241,0.291561,0.423300,-0.128296,-0.356305,1.041110,-0.187766,...,-0.508505,-0.615833,-1.028048,-1.885959,-0.236063,-0.217306,0.178173,0.222126,0.403050,0.280362
4,my grandad gave me this dvd. with friends like...,negative,-0.405184,0.649631,0.279136,-0.155268,0.251620,-0.550647,0.702823,0.108733,...,-0.822756,-1.015221,-1.796815,-1.531243,0.132349,-0.798254,0.144982,1.015178,0.292243,0.109142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,this film is like an allegory of the gospel. i...,positive,-0.270851,0.873439,0.166787,-0.160738,-0.120837,-0.399914,0.479787,0.136377,...,0.207984,-0.670058,-0.714821,-1.761524,0.242018,-0.422813,-0.045292,0.398858,0.522054,0.147394
3336,i saw dark harbor at the '98 seattle film fest...,positive,-0.268881,0.344730,0.906171,0.357031,0.282193,-1.045661,0.444265,0.170705,...,-0.368792,-1.560464,-1.190273,-1.104539,-0.047258,-0.755146,0.480339,0.163019,0.172857,0.193783
3337,i caught this film at a test screening. was ve...,positive,-0.663393,0.393591,0.234907,0.252559,-0.249900,-1.094168,0.409632,-0.081745,...,-0.504580,-1.770505,-1.282442,-2.228102,-0.111413,-0.380618,-0.043618,0.152464,0.106496,-0.088456
3338,an absolute classic of 80's scare flix. this o...,positive,-0.138735,0.504550,0.213303,-0.007190,-0.106794,-0.810001,1.151615,-0.010085,...,-0.844704,-0.409915,-1.552949,-1.404356,-0.198875,-0.813027,0.155421,0.118743,0.892833,0.074250


In [71]:
modelReadDataset = sampled_df.drop(['review'], axis=1)
modelReadDataset['sentiment'] = modelReadDataset['sentiment'].astype('category')
modelReadDataset

,sentiment,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,negative,0.273327,0.737564,0.740481,0.235589,-0.549669,-1.168450,1.045666,0.050999,0.645167,...,-0.504865,-0.335349,-1.372853,-2.010983,-0.313539,-0.939627,0.359824,0.658613,0.300944,0.361359
1,negative,-0.492114,1.152794,-0.250474,0.139126,-0.032523,-0.892238,1.060019,0.233915,0.506790,...,-0.207601,-0.501395,-1.164164,-1.465709,0.172248,-0.187147,0.014653,0.056565,0.893500,0.207951
2,negative,-0.130380,0.660078,0.567430,0.237133,-0.103683,-0.940961,1.543659,-0.179284,0.663818,...,-0.647385,-0.845036,-1.175116,-1.531967,-0.392439,-0.758413,-0.250309,0.167722,-0.209265,-0.121306
3,negative,-0.061912,0.318241,0.291561,0.423300,-0.128296,-0.356305,1.041110,-0.187766,0.619730,...,-0.508505,-0.615833,-1.028048,-1.885959,-0.236063,-0.217306,0.178173,0.222126,0.403050,0.280362
4,negative,-0.405184,0.649631,0.279136,-0.155268,0.251620,-0.550647,0.702823,0.108733,0.383937,...,-0.822756,-1.015221,-1.796815,-1.531243,0.132349,-0.798254,0.144982,1.015178,0.292243,0.109142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,positive,-0.270851,0.873439,0.166787,-0.160738,-0.120837,-0.399914,0.479787,0.136377,0.223936,...,0.207984,-0.670058,-0.714821,-1.761524,0.242018,-0.422813,-0.045292,0.398858,0.522054,0.147394
3336,positive,-0.268881,0.344730,0.906171,0.357031,0.282193,-1.045661,0.444265,0.170705,0.470950,...,-0.368792,-1.560464,-1.190273,-1.104539,-0.047258,-0.755146,0.480339,0.163019,0.172857,0.193783
3337,positive,-0.663393,0.393591,0.234907,0.252559,-0.249900,-1.094168,0.409632,-0.081745,0.558775,...,-0.504580,-1.770505,-1.282442,-2.228102,-0.111413,-0.380618,-0.043618,0.152464,0.106496,-0.088456
3338,positive,-0.138735,0.504550,0.213303,-0.007190,-0.106794,-0.810001,1.151615,-0.010085,0.378733,...,-0.844704,-0.409915,-1.552949,-1.404356,-0.198875,-0.813027,0.155421,0.118743,0.892833,0.074250


## Classify the text after converting the response variable into a categorical type

In [72]:
# Create a decision tree to classify sentiment using the remaining columns of modelReadDataset

## Drop the num_words column from modelReadDataset
# modelReadDataset.drop('num_words', axis=1, inplace=True)

# Rename the num_words column to 761
# modelReadDataset.rename(columns={'num_words': '761'}, inplace=True)

from sklearn.model_selection import train_test_split

X = modelReadDataset.drop('sentiment', axis=1)
y = modelReadDataset['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [73]:
from sklearn.tree import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier(random_state=42)

dt_classifier.fit(X_train, y_train)


DecisionTreeClassifier(random_state=42)

In [74]:
from sklearn import metrics

y_pred = dt_classifier.predict(X_test)

print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
# print('Precision:z', metrics.precision_score(y_test, y_pred))
# print('Recall:', metrics.recall_score(y_test, y_pred))
# print('F1 Score:', metrics.f1_score(y_test, y_pred))


Accuracy: 0.7125748502994012


In [75]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred, target_names=['negative', 'positive'])
print(report)


              precision    recall  f1-score   support

    negative       0.73      0.73      0.73       352
    positive       0.70      0.69      0.70       316

    accuracy                           0.71       668
   macro avg       0.71      0.71      0.71       668
weighted avg       0.71      0.71      0.71       668



## Instead of training a decision tree, lets train a simple fully-connected neural network to do the same task and see if it performs better or worse than the decision tree

In [77]:
modelReadDataset.shape

(3340, 769)

In [76]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assume X_train is your feature matrix and Y_train is your target variable

from sklearn.model_selection import train_test_split

X = modelReadDataset.drop('sentiment', axis=1)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(modelReadDataset['sentiment'])

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential()

# Add the first hidden layer
model.add(Dense(256, activation='relu', input_shape=(768,)))

# Add the second hidden layer
model.add(Dense(128, activation='relu'))

# Add the third hidden layer
model.add(Dense(64, activation='relu'))

# Add the output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Fit the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, verbose=1)


Epoch 1/100
84/84 [==============================] - 2s 4ms/step - loss: 0.4711 - accuracy: 0.7833
Epoch 2/100
84/84 [==============================] - 0s 4ms/step - loss: 0.4201 - accuracy: 0.7975
Epoch 3/100
84/84 [==============================] - 0s 4ms/step - loss: 0.4113 - accuracy: 0.8080
Epoch 4/100
84/84 [==============================] - 0s 4ms/step - loss: 0.3820 - accuracy: 0.8226
Epoch 5/100
84/84 [==============================] - 0s 4ms/step - loss: 0.3524 - accuracy: 0.8387
Epoch 6/100
84/84 [==============================] - 0s 4ms/step - loss: 0.3417 - accuracy: 0.8522
Epoch 7/100
84/84 [==============================] - 1s 6ms/step - loss: 0.3171 - accuracy: 0.8563
Epoch 8/100
84/84 [==============================] - 1s 8ms/step - loss: 0.2954 - accuracy: 0.8724
Epoch 9/100
84/84 [==============================] - 1s 6ms/step - loss: 0.2873 - accuracy: 0.8806
Epoch 10/100
84/84 [==============================] - 1s 7ms/step - loss: 0.2512 - accuracy: 0.8971
Epoch 11/

In [78]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 256)               196864    
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dense_23 (Dense)            (None, 1)                 65        
                                                                 
Total params: 238,081
Trainable params: 238,081
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Assume that X_test is your new dataset
predictions = model.predict(X_test)

# Convert probabilities into class labels
class_predictions = ['positive' if x > 0.5 else 'negative' for x in predictions]

# class_predictions

21/21 [==============================] - 0s 2ms/step


In [80]:
from sklearn.metrics import classification_report

report = classification_report(Y_test, le.fit_transform(class_predictions), target_names=['negative', 'positive'])
print(report)


              precision    recall  f1-score   support

    negative       0.79      0.80      0.79       352
    positive       0.77      0.76      0.77       316

    accuracy                           0.78       668
   macro avg       0.78      0.78      0.78       668
weighted avg       0.78      0.78      0.78       668



In [81]:
from sklearn import metrics

print('Accuracy:', metrics.accuracy_score(Y_test, le.fit_transform(class_predictions)))

Accuracy: 0.781437125748503


In [82]:
## Save dt_classifier to a pkl file for loading later
import pickle

with open('/content/gdrive/MyDrive/PGSS2023/Week 02/keras_mlp_model.pkl', 'wb') as f:
    pickle.dump(model, f)
